<a href="https://colab.research.google.com/github/patelhiren3385/scraping_data/blob/master/Scrapping_Placement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install scrapy
!pip3 install df2gspread
!pip install --upgrade google-api-python-client
!pip install authlib

In [ ]:
import requests
import bs4
import scrapy
from scrapy.crawler import CrawlerProcess
import json
import pandas as pd
import gspread
from df2gspread import df2gspread as d2g
from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build
from google.oauth2 import service_account
from google.auth.transport.requests import AuthorizedSession

In [ ]:
headers = {
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.102 Safari/537.36'
}
header_cord = {
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.102 Safari/537.36'
}

In [ ]:
login_data = {
    'rollno': '*********',
    'pass': '*********',
    'submit': 'Login'
}
login_data_cord = {
    'rollno': '*********',
    'pass': '*********',
    'submit': 'Login'
}


In [ ]:
with requests.Session() as s:
  url = 'https://pla*********'
  r = s.get(url,headers = headers)
  soup = bs4.BeautifulSoup(r.content, 'html')
  r = s.post(url,data = login_data, headers = headers)
  r = s.get('https://*********/comp_list_all.php',headers = headers)
  soup = bs4.BeautifulSoup(r.content, 'html')
  r = s.post('https://*********/comp_list_all.php',data = login_data, headers = headers)
  print(r.content)

b'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.1//EN" "http://www.w3.org/TR/xhtml11/DTD/xhtml11.dtd">\r\n<html xmlns="http://www.w3.org/1999/xhtml" xml:lang="en">\r\n\r\n<head>\r\n  <title>Training and Placement - IIT Madras</title>\r\n  <meta name="description" content="free website template" />\r\n  <meta name="keywords" content="enter your keywords here" />\r\n  <meta http-equiv="content-type" content="text/html; charset=utf-8" />\r\n  <meta http-equiv="X-UA-Compatible" content="IE=9" />\r\n  <link rel="stylesheet" type="text/css" href="../css/style.css" />\r\n  <script type="text/javascript" src="../js/jquery.min.js"></script>\r\n  <script type="text/javascript" src="../js/image_slide.js"></script>\r\n<SCRIPT language=JavaScript>\r\nfunction OpenPopup (c) {\r\n\t\twindow.open(c,\' \',\'width=620,height=900,scrollbars=yes\');\t\t\r\n\t\t}\r\nfunction OpenPopup1 (c) {\r\n\t\twindow.open(c,\' \',\'width=750,height=500,scrollbars=yes\');\t\t\r\n\t\t}\r\nfunction OpenPopup2 (c) {\r\n\t\tw

In [ ]:
r.close() 

In [ ]:
soup2 = bs4.BeautifulSoup(r.text, 'lxml')

In [ ]:
table = soup2.find_all() 
df = pd.read_html(str(table))
data = df[3]

In [ ]:
column = [data.loc[0][x] for x in range(7)]

In [ ]:
data.drop(0,inplace = True)
data.columns = column
data.set_index('Sl.No')

In [ ]:
data.drop('GD Date', inplace = True, axis = 1)

In [ ]:
data

In [ ]:
%cd /content/drive/My\ Drive/Placement_profile

/content/drive/My Drive/Placement_profile


In [ ]:
!ls

 client_secret.json
 client_secrets.json
'Copy of List of Companies for Applied Mechanics for 2020 placement_Web_Scrapped.gsheet'
'Copy of List of Companies for Applied Mechanics for 2020 placement_Web_Scrapped.xlsx'
 profiles.xlsx


In [ ]:
import json
from authlib.integrations.requests_client import AssertionSession

def create_assertion_session(conf_file, scopes, subject=None):
    with open(conf_file, 'r') as f:
        conf = json.load(f)

    token_url = conf['token_uri']
    issuer = conf['client_email']
    key = conf['private_key']
    key_id = conf.get('private_key_id')

    header = {'alg': 'RS256'}
    if key_id:
        header['kid'] = key_id

    # Google puts scope in payload
    claims = {'scope': ' '.join(scopes)}
    return AssertionSession(
        grant_type=AssertionSession.JWT_BEARER_GRANT_TYPE,
        token_endpoint=token_url,
        issuer=issuer,
        audience=token_url,
        claims=claims,
        subject=subject,
        key=key,
        header=header,
    )

scopes = [
    'https://spreadsheets.google.com/feeds',
    'https://www.googleapis.com/auth/drive',
]
session = create_assertion_session('client_secret.json', scopes)

In [ ]:
data.loc[1]

Sl.No                              1
Company Name           Tata Projects
Profile Name       Executive Trainee
PPT Date                          --
Resume Deadline           09/10/2020
Test Date                         --
Name: 1, dtype: object

In [ ]:
data

In [ ]:
from gspread import Client

gc = Client(None, session)

wks = gc.open("Copy of List of Companies for Applied Mechanics for 2020 placement_Web_Scrapped").sheet1

cell_list = wks.range('A2:A200')

# for cell,i in zip(cell_list,range(len(data))):
#   d = list(data.loc[i+1])
#   for j in range(6):
#     val = d[j]
#     # cell.value = val
#     wks.update_cell(i+1,j+1,val)

for cell,i in zip(cell_list,range(len(data))):
  cell.value = data['Resume Deadline'][i+1]
  wks.update_cell(i+1,1,val)

# Fetch a cell range
# cell_list = wks.range('A1:B7')

